In [20]:
from dataset import VoxDataset, ToTensor
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm_notebook as tqdm
from joblib import Parallel, delayed

In [21]:
dataset = VoxDataset(transform=ToTensor())

In [22]:
emb, mesh = dataset[0]

In [23]:
def get_max(emb, mesh):
    return max(emb), max(mesh)

In [24]:
result = Parallel(n_jobs=-1)(delayed(get_max)(emb, mesh) for emb, mesh in tqdm(dataset))

C:\Users\nawee\AppData\Local\Temp\ipykernel_24664\3608356375.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  result = Parallel(n_jobs=-1)(delayed(get_max)(emb, mesh) for emb, mesh in tqdm(dataset))


  0%|          | 0/107640 [00:00<?, ?it/s]

exception calling callback for <Future at 0x244b82b78d0 state=finished returned list>
Traceback (most recent call last):
  File "C:\Users\nawee\AppData\Roaming\Python\Python311\site-packages\joblib\parallel.py", line 1423, in dispatch_one_batch
    tasks = self._ready_batches.get(block=False)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nawee\.conda\envs\head\Lib\queue.py", line 168, in get
    raise Empty
_queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\nawee\.conda\envs\head\Lib\site-packages\pandas\core\indexes\range.py", line 413, in get_loc
    return self._range.index(new_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: 107640 is not in range

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\nawee\AppData\Roaming\Python\Python311\site-packages\joblib\externals\loky\_base.py", line 26, in _invoke_c

In [25]:
result

[(tensor(64.8758, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(86.8551, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(72.9523, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(72.4162, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(74.8109, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(73.2702, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(61.1574, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(77.2583, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(84.5400, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(77.1806, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(78.6262, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(85.1434, requires_grad=True), tensor(130.8786, requires_grad=True)),
 (tensor(72.4089, requires_grad=True), tensor(130.8786, requires

In [34]:
mesh_max = max([x[1] for x in result ]).detach()

In [35]:
emb_max = max([x[0] for x in result ]).detach()

In [36]:
emb_max

tensor(113.4636)

In [41]:
mesh_max

tensor(139.8999)

In [39]:
max(dataset[0][1]/mesh_max)

tensor(0.9355, grad_fn=<UnbindBackward0>)

In [40]:
max(dataset[0][0]/emb_max)

tensor(0.5718, grad_fn=<UnbindBackward0>)